In [ ]:
import subprocess, os
def run(cmd, cwd=None):
    p = subprocess.run(cmd, shell=True, cwd=cwd, capture_output=True, text=True)
    return p.returncode, p.stdout.strip(), p.stderr.strip()

print('Project root:', os.getcwd())
rc, node_v, node_err = run('node -v')
rc2, npm_v, npm_err = run('npm -v')
print('node:', node_v if rc==0 else f'not found ({node_err})')
print('npm: ', npm_v if rc2==0 else f'not found ({npm_err})')
nm_exists = os.path.exists(os.path.join('frontend','node_modules'))
print('frontend/node_modules exists:', nm_exists)
print('
If node or npm are missing, please install Node.js (18+).')

In [ ]:
import os, subprocess, time
cwd = os.path.join(os.getcwd(), 'frontend')
log = os.path.join(cwd, 'devserver.log')
os.makedirs(cwd, exist_ok=True)

def log_and_print(msg):
    print(msg)
    try:
        with open(log, 'a') as f:
            f.write(str(msg) + '\n')
    except Exception as e:
        print('Failed to write log:', e)

# Install deps if missing
if not os.path.exists(os.path.join(cwd, 'node_modules')):
    log_and_print('node_modules not found — running `npm ci` (may take several minutes)')
    try:
        p = subprocess.run(['npm','ci'], cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        with open(log, 'a') as f:
            f.write(p.stdout or '')
        log_and_print(f'npm ci exit code: {p.returncode}')
    except FileNotFoundError as e:
        log_and_print('npm not found; install Node.js (https://nodejs.org/)')
        raise

# Start dev server detached
log_and_print(f'Starting dev server (`npm run dev`) in {cwd}; logs -> {log}')
try:
    f = open(log, 'a')
    p = subprocess.Popen(['npm','run','dev'], cwd=cwd, stdout=f, stderr=f, start_new_session=True)
    log_and_print(f'Started PID: {p.pid}')
    log_and_print('Dev server URL: http://localhost:5173')
except Exception as e:
    log_and_print('Failed to start dev server: ' + str(e))
    raise